# Hybrid Machine Learning Model with Python

Hybrid machine learning models combine different types of algorithms to leverage their unique strengths, which results in improved predictive performance and robustness.

## When to build one?

Build a hybrid ML model when a single algorithm cannot capture data complexity. For example, use a hybrid approach when handling sequential patterns and broader trends in the data.

## Goals of this project

- Combine models like LSTM for sequence learning and Linear Regression for trend analysis to improve performance.

- Identify the need for a hybrid model when single model's perform poorly based on performance metrics.

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Projects/Hybrid Machine Learning Model/apple_stock_data.csv')
data.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Adj Close  252 non-null    float64
 2   Close      252 non-null    float64
 3   High       252 non-null    float64
 4   Low        252 non-null    float64
 5   Open       252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB


In [38]:
# Converting the date column into datetime format, setting it as index and focusing on the Close price
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]
data.head()

,Close
Date,
2023-11-02 00:00:00+00:00,177.570007
2023-11-03 00:00:00+00:00,176.649994
2023-11-06 00:00:00+00:00,179.229996
2023-11-07 00:00:00+00:00,181.820007
2023-11-08 00:00:00+00:00,182.889999


In [39]:
# Scaling the Close price between 0 and 1 using MinMaxScaler to ensure compatibility with the LSTM model

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))
data['Close'] = scaler.fit_transform(data[['Close']])
data.head()

,Close
Date,
2023-11-02 00:00:00+00:00,0.175853
2023-11-03 00:00:00+00:00,0.162983
2023-11-06 00:00:00+00:00,0.199077
2023-11-07 00:00:00+00:00,0.235311
2023-11-08 00:00:00+00:00,0.250280


In [40]:
data.head()

,Close
Date,
2023-11-02 00:00:00+00:00,0.175853
2023-11-03 00:00:00+00:00,0.162983
2023-11-06 00:00:00+00:00,0.199077
2023-11-07 00:00:00+00:00,0.235311
2023-11-08 00:00:00+00:00,0.250280


In [41]:
# Preparing the data for LSTM by creating sequences of a defined length (eg. 60 days) to predict the next day's price

import numpy as np

def create_sequences(data, seq_length = 60):
  X,y = [], [] # X -> input sequences, y -> target values
  for i in range(len(data) - seq_length):
    X.append(data[i:i+seq_length])
    y.append(data[i+seq_length])
  return np.array(X), np.array(y)

seq_length = 60
X,y = create_sequences(data['Close'].values, seq_length)

In [42]:
# Train test split
train_size = int(len(X)*0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [43]:
# Building a sequential LSTM model with layers to capture the temporal dependencies in the data

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Sequential =  A way to build a model layer by layer, where each layer feeds its output to the next one.
# LSTM = A type of neural network layer designed to handle sequential data (e.g., time-series data) and remember important patterns over time.
# Dense = A fully connected layer, typically used as the last layer to make predictions.

lstm_model = Sequential() # Initializes a Sequential model where layers will be added one by one.
lstm_model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))

# units = 50 => Adds an LSTM layer with 50 units (neurons). More neurons mean more capacity to learn complex patterns.
# return_sequences = True => This tells the layer to return the full sequence of outputs for each time step (not just the last output). This is necessary because another LSTM layer follows it.
# X_train.shape[1]: Number of time steps in each sequence (e.g., 60 if using seq_length=60).
# 1: Number of features per time step (e.g., just the "Close price")

lstm_model.add(LSTM(units = 50)) # For deeper learning
lstm_model.add(Dense(1)) # This is the final layer, and its output is the predicted value

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [44]:
# Compiling the model

lstm_model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Compile -> Prepares model for training
# Optimizer -> Determines how the model's weights are updated during training.
# Loss function -> Defines how the model's predictions are evaluated and how much the model's predictions deviate from the actual values.
# optimizer = 'adam' -> Adam is an optimization algorithm (method for adjusting weights) that is widely used because it works well with various types
# of models and data. It adjusts learning rates dynamically and helps the model converge faster.
# loss = 'mean_squared_error' -> In time-series prediction, MSE is often used for regression tasks like predicting stock prices.

lstm_model.fit(X_train,y_train, epochs = 20, batch_size = 32)

# epoch -> One complete pass through the entire training dataset.
# batch_size -> Number of training examples used in one iteration (before updating the model’s weights).

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0931
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0250
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0143
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0160
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0117
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - loss: 0.0110
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0091
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 0.0095
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - loss: 0.0091
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0086
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.0090
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.0081
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0082
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0083
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0082
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━

In [45]:
# Training the second model

# Generating lagged features for Linear Regression (eg. using past 3 days as predictors)

data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [46]:
len(data)

249

In [54]:
X_lin = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = data['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [48]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()
lin_model.fit(X_train_lin,y_train_lin)

LinearRegression()

In [49]:
# Making predictions using LSTM on the test set and inverse transforming the scaled predictions

X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 315ms/step


In [55]:
X_test_lin.shape

(96, 3)

In [61]:
X_test_lstm.shape

(39, 60, 1)

In [63]:
# Generating predictions using Linear Regression and inverse transforming them

lin_predictions = lin_model.predict(X_test_lin)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1,1))

In [51]:
lstm_predictions.shape

(39, 1)

In [52]:
lin_predictions.shape

(96, 1)

In [64]:
print("LSTM test set size:", X_test.shape, "and", y_test.shape)
print("Linear Regression test set size:", X_test_lin.shape, "and", y_test_lin.shape)

LSTM test set size: (39, 60) and (39,)
Linear Regression test set size: (96, 3) and (96,)


In [65]:
print("LSTM predictions shape:", lstm_predictions.shape)
print("Linear Regression predictions shape:", lin_predictions.shape)

LSTM predictions shape: (39, 1)
Linear Regression predictions shape: (96, 1)


In [66]:
if lstm_predictions.shape[0] > lin_predictions.shape[0]:
    lstm_predictions = lstm_predictions[:lin_predictions.shape[0]]
else:
    lin_predictions = lin_predictions[:lstm_predictions.shape[0]]

In [67]:
print("LSTM predictions shape:", lstm_predictions.shape)
print("Linear Regression predictions shape:", lin_predictions.shape)

LSTM predictions shape: (39, 1)
Linear Regression predictions shape: (39, 1)


In [68]:
# Using a weighted average to create hybrid predictions
hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)

# Giving 70% importance to LSTM predictions and 30% importance to the Linear Regression predictions in the final hybrid prediction.

In [70]:
# Making predictions for the next 10 days using hybrid model

lstm_future_predictions = []
last_sequence = X[-1].reshape(1, seq_length, 1)
for _ in range(10):
  lstm_pred = lstm_model.predict(last_sequence)[0,0]
  lstm_future_predictions.append(lstm_pred)
  lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1,1,1)
  last_sequence = np.append(last_sequence[:, 1:, :], lstm_pred_reshaped, axis = 1)
lstm_future_predictions = scaler.inverse_transform(np.array(lstm_future_predictions).reshape(-1,1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


In [71]:
# Predicting next 10 days using Linear Regression

recent_data = data['Close'].values[-3:]
lin_future_predictions = []
for _ in range(10):
    lin_pred = lin_model.predict(recent_data.reshape(1, -1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)
lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1, 1))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/

In [72]:
hybrid_future_predictions = (0.7 * lstm_future_predictions) + (0.3 * lin_future_predictions)

In [74]:
future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=10)
predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM Predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})
predictions_df.head()

,Date,LSTM Predictions,Linear Regression Predictions,Hybrid Model Predictions
0,2024-11-02 00:00:00+00:00,231.931381,230.355192,231.458517
1,2024-11-03 00:00:00+00:00,231.573669,225.707291,229.813750
2,2024-11-04 00:00:00+00:00,231.292969,222.703426,228.716103
3,2024-11-05 00:00:00+00:00,231.065079,230.631535,230.935020
4,2024-11-06 00:00:00+00:00,230.872498,225.486380,229.256662


In [76]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add LSTM predictions
fig.add_trace(go.Scatter(x=future_dates, y=lstm_future_predictions.flatten(), mode='lines+markers',
                         name='LSTM Predictions', line=dict(color='blue')))

# Add Linear Regression predictions
fig.add_trace(go.Scatter(x=future_dates, y=lin_future_predictions.flatten(), mode='lines+markers',
                         name='Linear Regression Predictions', line=dict(color='green')))

# Add Hybrid Model predictions
fig.add_trace(go.Scatter(x=future_dates, y=hybrid_future_predictions.flatten(), mode='lines+markers',
                         name='Hybrid Model Predictions', line=dict(color='red')))

fig.update_layout(
    title="Stock Price Predictions: LSTM vs Linear Regression vs Hybrid Model",
    xaxis_title="Date",
    yaxis_title="Stock Price",
    template="plotly_dark",  # Optional: Change the theme to dark
    xaxis=dict(tickformat="%Y-%m-%d", tickangle=45),  # Format x-axis as date and rotate the labels
    legend_title="Models",
    legend=dict(x=0.01, y=0.99)
)
fig.show()